In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score, roc_curve

In [51]:
matches = pd.read_csv("Football.csv")

/Users/ufukkose/Desktop/projects/soccer-predictor/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,71,72) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
matches.head()

,Country,League,home_team,away_team,home_score,away_score,season_year,Date_day,Date_hour,first_half,...,Distance_Covered_(km)_Home,Distance_Covered_(km)_Host,Clearances_Completed_Home,Clearances_Completed_Host,Pass_Success_per_Home,Pass_Success_per_Host,referee,venue,capacity,attendance
0,Germany,Bundesliga,B. Monchengladbach,Werder Bremen,4,1,2024/2025,3.11,19:30,3 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Dankert B. (Ger),Borussia-Park (Mönchengladbach),54 042,54 042
1,Germany,Bundesliga,Freiburg,Mainz,0,0,2024/2025,3.11,17:30,0 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Brand B. (Ger),Europa-Park Stadion (Freiburg),34 700,34 400
2,Germany,Bundesliga,Dortmund,RB Leipzig,2,1,2024/2025,2.11,20:30,1 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Stieler T. (Ger),Signal Iduna Park (Dortmund),81 365,81 365
3,Germany,Bundesliga,Bayern Munich,Union Berlin,3,0,2024/2025,2.11,17:30,2 - 0,...,NaN,NaN,NaN,NaN,NaN,NaN,Jollenbeck M. (Ger),Allianz Arena (Munich),75 024,75 000
4,Germany,Bundesliga,Eintracht Frankfurt,Bochum,7,2,2024/2025,2.11,17:30,4 - 1,...,NaN,NaN,NaN,NaN,NaN,NaN,Schlager D. (Ger),Deutsche Bank Park (Frankfurt),58 000,58 000


In [60]:
matches.shape

(95384, 91)

In [61]:
db = matches[(matches["League"] == "Bundesliga") & 
             (matches["season_year"].isin(["2023/2024", "2022/2023"]))].copy()
db.shape

(616, 91)

In [62]:
db["home_team"] = db["home_team"].str.replace(r"\n2", "", regex=True).str.strip()
db["away_team"] = db["away_team"].str.replace(r"\n2", "", regex=True).str.strip()
print(db)

     Country      League            home_team      away_team home_score  \
81   Germany  Bundesliga           Dusseldorf         Bochum          0   
82   Germany  Bundesliga               Bochum     Dusseldorf          0   
83   Germany  Bundesliga     Bayer Leverkusen       Augsburg          2   
84   Germany  Bundesliga             Dortmund      Darmstadt          4   
85   Germany  Bundesliga  Eintracht Frankfurt     RB Leipzig          2   
..       ...         ...                  ...            ...        ...   
692  Germany  Bundesliga   B. Monchengladbach     Hoffenheim          3   
693  Germany  Bundesliga               Bochum          Mainz          1   
694  Germany  Bundesliga         Union Berlin  Hertha Berlin          3   
695  Germany  Bundesliga            Wolfsburg  Werder Bremen          2   
696  Germany  Bundesliga  Eintracht Frankfurt  Bayern Munich          1   

    away_score season_year  Date_day Date_hour first_half  ...  \
81           4   2023/2024     27

In [63]:
null_vals = db.isnull().sum()
print("There are " + str(null_vals.sum()) + " missing values in the insurance csv.")

There are 14629 missing values in the insurance csv.


In [64]:
null_columns = db.isnull().sum()
null_columns = null_columns[null_columns > 0]
#print(null_columns)

In [65]:
print(db["home_team"].value_counts())
print(db["away_team"].value_counts())

Bochum                 35
Stuttgart              35
Freiburg               34
B. Monchengladbach     34
Mainz                  34
Eintracht Frankfurt    34
Union Berlin           34
Augsburg               34
FC Koln                34
Bayer Leverkusen       34
Wolfsburg              34
Hoffenheim             34
Dortmund               34
RB Leipzig             34
Werder Bremen          34
Bayern Munich          34
Darmstadt              17
Schalke                17
Heidenheim             17
Hertha Berlin          17
Hamburger SV            1
Dusseldorf              1
Name: home_team, dtype: int64
Stuttgart              35
Bochum                 35
B. Monchengladbach     34
Mainz                  34
Eintracht Frankfurt    34
Union Berlin           34
Augsburg               34
Freiburg               34
FC Koln                34
Bayer Leverkusen       34
Werder Bremen          34
Hoffenheim             34
Dortmund               34
RB Leipzig             34
Wolfsburg              34
Bayern M

In [72]:
db = db[~db["home_team"].isin(["Hamburger SV", "Dusseldorf"]) &
        ~db["away_team"].isin(["Hamburger SV", "Dusseldorf"])]
print(db["home_team"].value_counts())
print(db["away_team"].value_counts())

Freiburg               34
Bayer Leverkusen       34
Mainz                  34
Eintracht Frankfurt    34
Bochum                 34
Union Berlin           34
Augsburg               34
Stuttgart              34
FC Koln                34
B. Monchengladbach     34
Hoffenheim             34
Dortmund               34
RB Leipzig             34
Werder Bremen          34
Bayern Munich          34
Wolfsburg              34
Darmstadt              17
Schalke                17
Heidenheim             17
Hertha Berlin          17
Name: home_team, dtype: int64
Stuttgart              34
Bayer Leverkusen       34
Mainz                  34
Eintracht Frankfurt    34
Bochum                 34
Union Berlin           34
Augsburg               34
Freiburg               34
FC Koln                34
B. Monchengladbach     34
Hoffenheim             34
Dortmund               34
RB Leipzig             34
Wolfsburg              34
Bayern Munich          34
Werder Bremen          34
Darmstadt              17
Schalke 

In [75]:
nunique = db.nunique()
constant_cols = nunique[nunique == 1].index.tolist()
print("Constant columns:", constant_cols)

Constant columns: ['Country', 'League', 'Tackles_Home', 'Tackles_Host']


In [76]:
db = db.drop(columns=constant_cols)

In [81]:
null_ratio = db.isnull().mean()
high_null_cols = null_ratio[null_ratio > 0.5].index.tolist()
print("Columns with too many null values:", high_null_cols)

Columns with too many null values: ['home_team_red_card_current_time', 'home_team_red_card', 'home_team_red_card_why', 'away_team_red_card_current_time', 'away_team_red_card', 'away_team_red_card_why', 'Blocked_Shots_Home', 'Blocked_Shots_Host', 'Red_Cards_Home', 'Red_Cards_Host', 'Crosses_Completed_Home', 'Crosses_Completed_Host', 'Interceptions_Home', 'Interceptions_Host', 'Distance_Covered_(km)_Home', 'Distance_Covered_(km)_Host', 'Clearances_Completed_Home', 'Clearances_Completed_Host', 'Pass_Success_per_Home', 'Pass_Success_per_Host']


In [82]:
db = db.drop(columns=high_null_cols)

In [85]:
db.shape

(612, 67)